# Introduction to TensorFlow

## Introduction
In this tutorial, we will introduce the TensorFlow to you by using it to train two models to predict if a bank should approve the requests for the bank's credit card based several possibly relevant credit records. We will mainly focus on the TF.Learn API in TensorFlow in this tutorial. In the first part we will use a simple linear model in TensorFlow for this problem. And in the second part, we will train a deep neural network model instead.

I hope this tutorial could help you get more familiarized with some machine learning concepts in TensorFlow as well as its APIs and data structures. And more importantly, have fun with TensorFlow!

## Setup
Firstly, you need to choose a correct version of TensorFlow (for both your computer and your Python version). Check out the version list [here](https://www.tensorflow.org/versions/r0.11/get_started/os_setup.html#pip-installation). After you download the right version, you could use pip to install the TensorFlow directly.

To test your installation, you could use the following hello world program.

In [1]:
import tensorflow as tf
sess = tf.Session()
hello = tf.constant('Hello, TensorFlow!')
print(sess.run(hello))
a = tf.constant(10)
b = tf.constant(32)
print(sess.run(a + b))

Hello, TensorFlow!
42


## Linear Model with TensorFlow
The first part of this tutorial is to solve a binary classification problem with a linear classification model as I mentioned above. And firstly, please import the libraries below before we move on. And to prevent too many warning logs when running the program later, I suggest you to set the logging verbosity to ERROR.

In [2]:
import tensorflow as tf
import pandas as pd
import os
import urllib
tf.logging.set_verbosity(tf.logging.ERROR)

### Load Census Data
The first step is to load training and test data from an income census. Here we download the data from [this link](https://archive.ics.uci.edu/ml/datasets/Credit+Approval). This dataset is interesting because there is a good mix of attributes -- continuous, nominal with small numbers of values, and nominal with larger numbers of values. There are also a few missing values.

In [3]:
if not os.path.isfile('./credit.csv'):
    urllib.urlretrieve('https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data', 'credit.csv')
    pass
credit_file = open("credit.csv", "r")

Now let's take a look at the data we have before processing on them. Here is a table of all attributes of different columns in the csv file.

| Column Name | Type | Description |
| ----------- | ---- | ----------- |
| A1 | Categorical | Binary valued categories (str) |
| A2 | Continuous | Continuous large number (float) |
| A3 | Continuous | Continuous small number (float) |
| A4 | Categorical | Multiple valued categories (str) |
| A5 | Categorical | Multiple valued categories (str) |
| A6 | Categorical | Multiple valued categories (str) |
| A7 | Categorical | Multiple valued categories (str) |
| A8 | Continuous | Continuous small number (float) |
| A9 | Categorical | Binary valued categories (str) |
| A10 | Categorical | Binary valued categories (str) |
| A11 | Continuous | Continuous large number (float) |
| A12 | Categorical | Binary valued categories (str) |
| A13 | Categorical | Multiple valued categories (str) |
| A14 | Continuous | Continuous large number (float) |
| A15 | Continuous | Continuous large number (float) |
| A16 | Categorical | Binary valued labels (str). '+' means request approved, otherwise it will be '-' |

Notice that we even don't know most of the meanings of each attributes. But don't worry, with the magic of Data Science, the data will tell us everything. And here, TensorFlow will take care of everything, so all we need to do is to choose a correct model and train it with the data.

Here we just simply divide all attributes into two lists of different types of attributes.

In [4]:
COLUMNS = ["A1", "A2", "A3", "A4", "A5", "A6", "A7", "A8", "A9",
           "A10", "A11", "A12", "A13", "A14", "A15", "A16"]
CATEGORICAL_COLUMNS = ["A1", "A4", "A5", "A6", "A7", "A9", "A10", "A12", "A13"]
CONTINUOUS_COLUMNS = ["A2", "A3", "A8", "A11", "A14", "A15"]
LABEL_COLUMN = "label"

As what we did in other homeworks, we use pandas to load the data, from the csv files we just downloaded, into Pandas dataframes. Since there are incomplete values in the file represented by '?', we need to drop those rows with incomplete values. And we divide the whole dataframe into two dataframes. One for training and one for testing.

In [5]:
df_credit = pd.read_csv(credit_file, names=COLUMNS, skipinitialspace=True, na_values=['?'])
df_credit.dropna(how='any', inplace=True)
df_train = df_credit.head(520)
df_test = df_credit.tail(133)

### Process Raw Data
Next, we need to process on them. The first thing we will do is to create a label column for each instance of the data. Although there is a column called 'A16' which could be used to represent the label, we still need to transform it into a numeric field whose value is 1 if request can be approved, and 0 otherwise.

In [6]:
df_train.loc[:, LABEL_COLUMN] = df_train["A16"].apply(lambda x: 1 if x == "+" else 0)
df_test.loc[:, LABEL_COLUMN] = df_test["A16"].apply(lambda x: 1 if x == "+" else 0)

/Applications/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:288: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Applications/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Next, we are gonna transform the other raw data into the most important data structure in TensorFlow - Tensor. A Tensor is a symbolic handle to one of the outputs of an Operation. It does not hold the values of that operation's output, but instead provides a means of computing those values in a TensorFlow Session. There are mainly two purposes of using this class:
1. A Tensor can be passed as an input to another Operation. This builds a dataflow connection between operations, which enables TensorFlow to execute an entire Graph that represents a large, multi-step computation.
2. After the graph has been launched in a session, the value of the Tensor can be computed by passing it to Session.run(). t.eval() is a shortcut for calling tf.get_default_session().run(t).
Here, we will write a builder function to convert input data into Tensors.

In [7]:
def process(df):
    """ Given a dataframe of credit record data with labels representing if a credit card request should be
    approved, generate a dictionary mapping from each attribute name to the value of that attribute stored
    in Tensor or SparseTensor and a list of labels stored in Tensor.
    
    Args:
    df (dataframe): a dataframe of income census data.
    
    Return:
    X (dictionary): a dictionary mapping each attribute to the values of that attribute in Tensor or SparseTensor.
    Y (Tensor): a list of labels in Tensor
    """
    
    continuous_cols = {
        k: tf.constant(
            df[k].values,
            shape=[df[k].size, 1]
        ) for k in CONTINUOUS_COLUMNS
    }
    
    categorical_cols = {
        k: tf.SparseTensor(
            indices=[[i, 0] for i in range(df[k].size)],
            values=df[k].values,
            shape=[df[k].size, 1]
        ) for k in CATEGORICAL_COLUMNS
    }
    
    # Merges the two dictionaries into one.
    X = dict(continuous_cols.items() + categorical_cols.items())
    
    Y = tf.constant(
        df[LABEL_COLUMN].values,
        shape=[df[LABEL_COLUMN].size, 1]
    )
    
    return X, Y

    pass

There are two ways to pass the training data X, Y into TensorFlow Classifier's fit or evaluation function. One way is to make the input X and Y to be instances of built-in data structure Tensor and pass them directly to the fit or evaluation function. The other way is to pass a callable function to the fit function and this function should be able to return the input X and Y when TensorFlow calls it. So we write the two following simple functions. Later, we will pass these two functions to the linear classification model for training.

In [8]:
def train_input_fn():
    return process(df_train)


def eval_input_fn():
    return process(df_test)

### Make Soft Bias
Next, we need to make biases based on each attribute's data type. It is one of the most important step in machine learning - making biases.

Firstly we will use TF.Learn API to create SparseColumn for categorical attribute. Since we already know all possible values of these columns, we can make these columns to be sparse with keys.

In [9]:
a1 = tf.contrib.layers.sparse_column_with_keys(column_name="A1", keys=["b", "a"])
a9 = tf.contrib.layers.sparse_column_with_keys(column_name="A9", keys=["t", "f"])
a10 = tf.contrib.layers.sparse_column_with_keys(column_name="A10", keys=["t", "f"])
a12 = tf.contrib.layers.sparse_column_with_keys(column_name="A12", keys=["t", "f"])

One of the good things in TensorFlow is that we don't necessarily need to know all of the possible values of an attribute in advance. With the help of TensorFlow API sparse_column_with_hash_bucket, we can transform the raw data easily with a hash function into feature columns. But to choose a proper hash bucket size, we at least need to know the upper bound of the number of possible values for these attributes.

In [10]:
a4 = tf.contrib.layers.sparse_column_with_hash_bucket("A4", hash_bucket_size=10)
a5 = tf.contrib.layers.sparse_column_with_hash_bucket("A5", hash_bucket_size=10)
a6 = tf.contrib.layers.sparse_column_with_hash_bucket("A6", hash_bucket_size=20)
a7 = tf.contrib.layers.sparse_column_with_hash_bucket("A7", hash_bucket_size=10)
a13 = tf.contrib.layers.sparse_column_with_hash_bucket("A13", hash_bucket_size=10)

And then, for the continuous attributes, we can directly use RealValuedColumn in TF.Learn.

In [11]:
a2 = tf.contrib.layers.real_valued_column("A2")
a3 = tf.contrib.layers.real_valued_column("A3")
a8 = tf.contrib.layers.real_valued_column("A8")
a11 = tf.contrib.layers.real_valued_column("A11")
a14 = tf.contrib.layers.real_valued_column("A14")
a15 = tf.contrib.layers.real_valued_column("A15")

### Linear Classification Model
Now we can move on to build a Linear Classification model with TensorFlow. With the help of TensorFlow, we only need several lines of code to build up a cool Linear Classification model and save it to your local file system. And the model can automatically learn the bias terms, which controls the prediction without observing any features.

Training is very easy in TensorFlow. We only need one line of code to train the model with our processed data in iterations. Notice that the steps parameter here is just the iteration number we used in our homework. You can change the steps here to any value you want. Here we set the iteration number to 100. But since our model is saved to the local file system, you can keep training it again and again and see the performance of our model getting better and better.

And after we have the trained model L_m. We can evaluate this model with the test data that we partitioned from the downloaded data at the very beginning. TensorFlow also makes the evaluation process very easy. The steps parameter is just like the steps parameter in the fit function.

In [12]:
if not os.path.exists('linear_model_dir'):
    os.makedirs('linear_model_dir')
    pass

L_m = tf.contrib.learn.LinearClassifier(
    feature_columns=[a1, a2, a3, a4, a5, a6, a7,
                     a8, a9, a10, a11, a12, a13, a14, a15],
    model_dir='linear_model_dir',
    enable_centered_bias=True
)

L_m.fit(input_fn=train_input_fn, steps=100)

results = L_m.evaluate(input_fn=eval_input_fn, steps=1)
for key in sorted(results):
    print "%s: %s" % (key, results[key])
    pass

accuracy: 0.774436
accuracy/baseline_target_mean: 0.390977
accuracy/threshold_0.500000_mean: 0.774436
auc: 0.842474
global_step: 200
labels/actual_target_mean: 0.390977
labels/prediction_mean: 0.401459
loss: 1.37965
precision/positive_threshold_0.500000_mean: 0.711538
recall/positive_threshold_0.500000_mean: 0.711538


From the results printed out above, you should be able to see that the accuracy is around 77.4% which is a fairly good accuracy for the linear model. Next, you should be able to predict if a request for credit card should be approved or not with this model L_m, by calling L_m.predict(x=x). You can adjust the steps or other parameters in fit and evaluate function above to see different results here.

## Neural Network Model with TensorFlow
Well, you might think a linear model here is not good enough to solve this problem and maybe we can do better with more complicated models. Yes! Actually, as a modern machine learning framework, TensorFlow is very good at training deep neural network as well. So let's just get started building this deep neural network.

### Build Embedding Vectors
Before training the neural network, our first task is to convert the categorical sparse columns we get in the first part into embedding vectors for the neural network model in TensorFlow. Because the feature columns used in the deep neural network in TensorFlow need to be dense and low-dimensional, which means the original sparse columns will be compressed to dense vectors before the actual training. And the dense, low-dimensional columns will later be fed into the hidden layers during training. And here, since continuous columns cannot be compressed, we concatenate them directly with the embedding vectors.

To achieve better performance, we need to set the dimension parameters here properly according to how well you want the original sparse data to be compressed. Since most of the original data are hashed into sparse columns with a bucket size. We can use the bucket size to determine how many dimensions we want here. Generally, the relationship between bucket size and dimensions are log based. What I did here was mapping bucket size 10 to dimension 4 and bucket size 20 to dimension 5.

In [13]:
deep_columns = [
    tf.contrib.layers.embedding_column(a1, dimension=2),
    tf.contrib.layers.embedding_column(a4, dimension=4),
    tf.contrib.layers.embedding_column(a5, dimension=4),
    tf.contrib.layers.embedding_column(a6, dimension=5),
    tf.contrib.layers.embedding_column(a7, dimension=4),
    tf.contrib.layers.embedding_column(a9, dimension=2),
    tf.contrib.layers.embedding_column(a10, dimension=2),
    tf.contrib.layers.embedding_column(a12, dimension=2),
    tf.contrib.layers.embedding_column(a13, dimension=4),
    a2, a3, a8, a11, a14, a15
]

### Deep Neural Network Model
Now we have everything necessary for the training and are ready to build the neural network model. We can just use the Deep Neural Network class provided by TensorFlow to build our training model. Again, everything of this model can be saved to your local file system so that your training time on this model won't be wasted.

Notice, we still need to specify the number of perceptron units in the hidden layer and the number of layers. To make the training process faster, here I only use three hidden layers each with 256, 128, 64 perceptron units.

In [14]:
if not os.path.exists('DNN_model_dir'):
    os.makedirs('DNN_model_dir')
    pass

DNN_m = tf.contrib.learn.DNNClassifier(
    model_dir='DNN_model_dir',
    feature_columns=deep_columns,
    hidden_units=[256, 128, 64],
    enable_centered_bias=True
)

DNN_m.fit(input_fn=train_input_fn, steps=100)

results = DNN_m.evaluate(input_fn=eval_input_fn, steps=1)
for key in sorted(results):
    print "%s: %s" % (key, results[key])
    pass

accuracy: 0.699248
auc: 0.700499
global_step: 200
loss: 0.608915


For your references, the accuracy for this neural network is around 70.7% which is even much worse than the linear model. Why does this happen? It is because the dense embeddings themselves cannot generate zero predictions for all attributes. In this case, dense embeddings can over-generalize the output.

### Combine Linear Model and Neural Network Model
So how to solve this problem? Fortunately, TensorFlow provide us a way to combine the linear model we used in the first part and the neural network model above to create a combination of these two. Next, we will use the Deep Neural Network and Linear Combined Classifier class in TensorFlow for training on both linear attributes and deep columns.

In [15]:
if not os.path.exists('DNN_Linear_model_dir'):
    os.makedirs('DNN_Linear_model_dir')
    pass

DNN_Linear_m = tf.contrib.learn.DNNLinearCombinedClassifier(
    model_dir='DNN_Linear_model_dir',
    linear_feature_columns=[a1, a2, a3, a4, a5, a6, a7, 
                            a8, a9, a10, a11, a12, a13, a14, a15],
    dnn_feature_columns=deep_columns,
    dnn_hidden_units=[256, 128, 64]
)

DNN_Linear_m.fit(input_fn=train_input_fn, steps=2000)
results = DNN_Linear_m.evaluate(input_fn=eval_input_fn, steps=1)
for key in sorted(results):
    print "%s: %s" % (key, results[key])
    pass

accuracy: 0.87218
accuracy/baseline_target_mean: 0.390977
accuracy/threshold_0.500000_mean: 0.87218
auc: 0.898742
global_step: 4000
labels/actual_target_mean: 0.390977
labels/prediction_mean: 0.350709
loss: 0.514521
precision/positive_threshold_0.500000_mean: 0.87234
recall/positive_threshold_0.500000_mean: 0.788462


This time, as you can see, the accuracy is improved a lot from 70% to almost 90%. Again, you can change the parameters to see different results. Generally, the Combination of Linear Model and Deep Neural Network can always achieve better performance than either one of them with enough training data and number of iterations.

## Conclusion
So this is pretty much everything I want to talk about the TensorFlow. Notice, this is just a very little piece of all the functionalities of this great framework. TensorFlow can be used to do many other machine learning tasks and it is still growing stronger. This tutorial focuses only on the basic usage of several common machine learning models in TensorFlow and some basic data structures in TensorFlow. Hope you would like it and enjoying playing with TensorFlow.